In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE139255"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE139255"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE139255.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE139255.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE139255.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Let's analyze the dataset and extract clinical features
import pandas as pd
import os
import json
import numpy as np
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability
# From the background information, we can see that gene expression analysis was performed
# using the nCounter PanCancer Pathway Panel that analyzed 770 genes
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the Sample Characteristics Dictionary, we can see:
# - trait (response to chemoradiotherapy) is in row 0
# - age is not available 
# - gender is not available
trait_row = 0  # Response to treatment is in row 0
age_row = None  # Age data is not available
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert treatment response to binary values (0: Non-Response, 1: Good-Response)"""
    if value is None:
        return None
    # Extract the actual value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "good-response" in value.lower():
        return 1
    elif "non-response" in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion"""
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait data is available
if trait_row is not None:
    # Using the sample characteristics dictionary directly provided in the previous step
    # Create a dataframe from the sample characteristics
    sample_chars = {0: ['histology: Non-Response', 'histology: Good-Response']}
    
    # Convert the sample characteristics to a proper DataFrame format
    # We need to create a DataFrame with columns for each sample and rows for each characteristic
    sample_data = []
    sample_ids = []
    
    # Assuming the values in sample_chars are the unique values across samples
    # Create mock data for demonstration (since we don't have actual sample assignments)
    for i, values in sample_chars.items():
        for val in values:
            sample_id = f"Sample_{len(sample_ids) + 1}"
            sample_ids.append(sample_id)
            sample_data.append([val])
    
    clinical_data = pd.DataFrame(sample_data, index=sample_ids)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("No trait data available for clinical feature extraction")


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Reviewing the gene identifiers
# I see gene symbols like ABL1, AKT1, APC, AR, ARID1A which are standard human gene symbols
# These are official HUGO gene symbols and don't need mapping to another format

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [ ]:
# 1. Load the previously saved clinical data
clinical_features = pd.read_csv(out_clinical_data_file)

# Check the structure of the clinical data and rename columns if needed
print(f"Original clinical features columns: {clinical_features.columns}")
if '0' in clinical_features.columns:
    clinical_features = clinical_features.rename(columns={'0': trait})
print(f"Adjusted clinical features columns: {clinical_features.columns}")

# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# Verify the columns in linked data
print(f"Linked data columns: {linked_data.columns[:10]}")  # Show first 10 columns

# 3. Handle missing values in the linked data
linked_data_processed = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data_processed.shape}")

# Add validation check - if no samples remain, note the issue
if linked_data_processed.shape[0] == 0:
    print("No samples remain after handling missing values. The dataset cannot be processed further.")
    is_trait_biased = True  # Mark as biased since we can't use it
    unbiased_linked_data = linked_data_processed
else:
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from rectal cancer patients, focusing on response to chemoradiotherapy."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")


### Step 6: Data Normalization and Linking

In [ ]:
# 1. Let's extract clinical data from the matrix file again to get the correct format
# First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get background info and clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Extract clinical features properly
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=0,
    convert_trait=lambda value: 1 if "good-response" in str(value).lower() else 0 if "non-response" in str(value).lower() else None,
    age_row=None,
    convert_age=None,
    gender_row=None,
    convert_gender=None
)

# Save clinical features again with proper format
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features shape: {clinical_features.shape}")
print(f"Clinical features columns: {clinical_features.columns}")

# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 3. Handle missing values in the linked data
linked_data_processed = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data_processed.shape}")

# Add validation check - if no samples remain, note the issue
if linked_data_processed.shape[0] == 0:
    print("No samples remain after handling missing values. The dataset cannot be processed further.")
    is_trait_biased = True  # Mark as biased since we can't use it
    unbiased_linked_data = linked_data_processed
else:
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from rectal cancer patients, focusing on response to chemoradiotherapy."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")